In [2]:
!pip install icecream

In [3]:
from io import StringIO 
import requests  # for load api
from bs4 import BeautifulSoup # for parsing HTML and XML documents
import pandas as pd # for transformation
import sqlite3 # for load data and make databases
from datetime import datetime # 
from icecream import icecream as ic 

In [5]:
x=datetime.now()  #current datatime 
print(x)

2024-11-26 01:19:34.848210


### step 0

In [6]:
def log_progress(message):
    with open('./code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}.\n')

### step 1

In [7]:
def extract(url,table_att):
    soup = requests.get(url).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span', string = table_att).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]

    log_progress('Data Extracted Successfully form the Wikipedia')

    return df

### step 2

In [8]:
def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col = 0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['GBP'],2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['EUR'],2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['INR'],2)

    print(df['MC_EUR_Billion'][4])
    log_progress('Data Transformation Successfully done, step 2')

    return df

### step 3

In [9]:
def load_to_csv(df, out_path):
    df.to_csv(out_path)

    log_progress('Data Loaded into csv, step 3')


### step 4

In [10]:
#sqlite3

def load_to_db(df, sql_conn, tn):
    df.to_sql(tn, sql_conn, if_exists='replace',index=False)

    log_progress('Data Loaded into sqlite3, step 4')

def run_query(query_statement, sql_connection):
    """ This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. """

    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_progress('Process Complete')

    return result

In [13]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')

    df = extract(url, 'By market capitalization')
    log_progress('Extraction complete')
    print(df)

    df = transformation(df, './datasets/exchange_rate.csv')
    log_progress('Extraction complete')
    print(df)

    load_to_csv(df, output_csv_path)
    log_progress('data has been loaded into csv')

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)   
        log_progress('data has been loaded into sqlite3')
        run_query('SELECT * FROM Largest_banks', conn)

        run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn)

        run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn)
        log_progress('Query run')
    
    

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2     